# 🎯 Data Preprocessing

In this notebook, we take a unified dataset `(The dataset contains hourly electricity demand and weather measurements for
ten major U.S. cities.)` for ten U.S. cities (already loaded  
and merged in a separate notebook for simplicity) and perform the following steps:

1. **Missing Values**  
   Identify and either impute or remove missing entries.

2. **Feature Engineering**  
   - Extract time-based features: hour, day of week, month, season  
   - Normalize/scale continuous variables to mean=0, std=1

3. **Aggregation**  
   Compute daily and weekly summary statistics for demand and key weather variables.

4. **Anomaly & Error Detection**  
   - Use z‑score and IsolationForest to flag outliers (e.g., sudden consumption spikes, impossible weather readings, sensor faults).  
   - Investigate and remove or correct the flagged anomalies.

By the end, we produce a `final_df` containing exactly the features needed for downstream modeling or analysis, and save it to an external drive.


In [41]:
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import StandardScaler
from scipy.stats import zscore
from sklearn.ensemble import IsolationForest
from sklearn.exceptions import DataConversionWarning

# Silence the scikit‑learn feature‑name warning
warnings.filterwarnings("ignore", category=DataConversionWarning)


#### 📂 Load Dataset and Initial Inspection

Read the CSV from the external drive, then print shape, head, dtypes,  
and info to understand columns and missingness.

In [42]:
# adjust path if needed
df_sample = pd.read_csv('/Volumes/meen/sampled_data.csv')

# Quick checks
print("Shape:", df_sample.shape)
display(df_sample.head())

# Column data types
for col, dt in df_sample.dtypes.items():
    print(f"{col}: {dt}")
print("\nDataFrame Info:")
df_sample.info()

# Missing values per column
print("\nMissing values per column:")
print(df_sample.isna().sum())


Shape: (50000, 25)


,company,local_time,utc_time,demand,city,summary,icon,precipIntensity,precipProbability,temperature,...,windGust,windBearing,cloudCover,uvIndex,visibility,precipType,ozone,precipAccumulation,region,period
0,ERCO,2018-07-03 17:00:00,2018-07-03 22:00:00,NaN,dallas,Clear,clear-day,0.0000,0.00,102.44,...,5.89,129.0,0.04,5.0,9.806,NaN,NaN,NaN,NCEN,2018-07
1,ERCO,2020-05-02 14:00:00,2020-05-02 19:00:00,13081.0,dallas,Clear,clear-day,0.0000,0.00,83.02,...,24.63,172.0,0.20,9.0,10.000,NaN,315.7,NaN,NCEN,2020-05
2,ERCO,2019-06-20 21:00:00,2019-06-21 02:00:00,21034.0,dallas,Humid,clear-night,0.0000,0.00,89.95,...,22.00,167.0,0.16,0.0,10.000,NaN,315.8,NaN,NCEN,2019-06
3,ERCO,2019-11-16 09:00:00,2019-11-16 15:00:00,14107.0,dallas,Clear,clear-day,0.0011,0.01,45.18,...,7.89,150.0,0.11,1.0,10.000,rain,283.7,NaN,NCEN,2019-11
4,ERCO,2019-03-30 12:00:00,2019-03-30 17:00:00,NaN,dallas,Overcast,cloudy,0.0009,0.12,54.59,...,26.30,330.0,0.96,4.0,10.000,rain,313.8,NaN,NCEN,2019-03


company: object
local_time: object
utc_time: object
demand: float64
city: object
summary: object
icon: object
precipIntensity: float64
precipProbability: float64
temperature: float64
apparentTemperature: float64
dewPoint: float64
humidity: float64
pressure: float64
windSpeed: float64
windGust: float64
windBearing: float64
cloudCover: float64
uvIndex: float64
visibility: float64
precipType: object
ozone: float64
precipAccumulation: float64
region: object
period: object

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   company              50000 non-null  object 
 1   local_time           50000 non-null  object 
 2   utc_time             50000 non-null  object 
 3   demand               42125 non-null  float64
 4   city                 50000 non-null  object 
 5   summary              49920 non-null  object 
 

#### 🗑️ Drop Irrelevant Columns

Remove metadata and overly‐sparse columns to streamline the dataframe

In [43]:
cols_to_remove = [
    'company','apparentTemperature','dewPoint','windGust','windBearing',
    'cloudCover','visibility','ozone','region','icon'
]
df_sample = df_sample.drop(columns=cols_to_remove, errors='ignore')

print("Remaining columns:", df_sample.columns.tolist())


Remaining columns: ['local_time', 'utc_time', 'demand', 'city', 'summary', 'precipIntensity', 'precipProbability', 'temperature', 'humidity', 'pressure', 'windSpeed', 'uvIndex', 'precipType', 'precipAccumulation', 'period']


#### ⏱️ Parse and Clean Timestamps

Convert whichever time column exists into a single `timestamp`,  then drop the originals.

In [44]:
if 'utc_time' in df_sample.columns:
    df_sample['timestamp'] = pd.to_datetime(df_sample['utc_time'])
    drop_cols = ['utc_time', 'local_time'] if 'local_time' in df_sample.columns else ['utc_time']
elif 'local_time' in df_sample.columns:
    df_sample['timestamp'] = pd.to_datetime(df_sample['local_time'])
    drop_cols = ['local_time']
else:
    drop_cols = []

df_sample = df_sample.drop(columns=drop_cols, errors='ignore')


#### Missing‐Value Imputation

1. Drop rows missing the target (`demand`).  
2. Impute continuous weather columns by **city‐median**, then global median.  
3. Fill categorical/text gaps with placeholders.

In [46]:
# 5.1 Drop rows with no demand
df_sample = df_sample[df_sample['demand'].notna()]

# 5.2 Continuous columns to impute
cont_cols = [
    'precipIntensity','precipProbability','temperature',
    'humidity','pressure','windSpeed','uvIndex',
    'precipAccumulation'
]
cont_cols = [c for c in cont_cols if c in df_sample.columns]

# City-median fill
for c in cont_cols:
    df_sample[c] = (df_sample
        .groupby('city')[c]
        .transform(lambda x: x.fillna(x.median()))
    )

# Global-median fill
for c in cont_cols:
    df_sample[c].fillna(df_sample[c].median(), inplace=True)

# Categorical/text fill
if 'summary' in df_sample.columns:
    df_sample['summary'] = df_sample['summary'].fillna('Unknown')
if 'precipType' in df_sample.columns:
    df_sample['precipType'] = df_sample['precipType'].fillna('none')


#### 🌱 6. Feature Engineering

Extract time‐based features (hour, day of week, month, season) and add them to the dataframe.

In [47]:
# Time components
df_sample['hour']      = df_sample['timestamp'].dt.hour
df_sample['dayofweek'] = df_sample['timestamp'].dt.dayofweek
df_sample['month']     = df_sample['timestamp'].dt.month

# Map month → season
def month_to_season(m):
    if m in [12,1,2]: return 'winter'
    if m in [3,4,5]:  return 'spring'
    if m in [6,7,8]:  return 'summer'
    return 'autumn'

df_sample['season'] = df_sample['month'].map(month_to_season)


#### ⚖️ Scale Continuous Variables

Normalize all continuous columns (including `demand`) to mean=0, std=1  
using `StandardScaler`.

In [48]:
scaler = StandardScaler()
to_scale = ['demand'] + cont_cols  # ensure demand + weather vars
df_sample[to_scale] = scaler.fit_transform(df_sample[to_scale])


#### 📊 Aggregation to Daily & Weekly Summaries

Demonstrate how to compute mean demand and weather stats per city/day and per city/week.

In [49]:
# Daily
df_sample['date'] = df_sample['timestamp'].dt.date
daily = (
    df_sample
    .groupby(['city','date'])
    .agg({
        'demand':'mean',
        'temperature':'mean',
        'humidity':'mean',
        'windSpeed':'mean'
    })
    .reset_index()
    .rename(columns={
        'demand':'demand_daily_avg',
        'temperature':'temp_daily_avg',
        'humidity':'hum_daily_avg',
        'windSpeed':'wind_daily_avg'
    })
)

# Weekly
df_sample['week'] = df_sample['timestamp'].dt.isocalendar().week
weekly = (
    df_sample
    .groupby(['city','week'])
    .agg({
        'demand':'mean',
        'temperature':'mean',
        'humidity':'mean'
    })
    .reset_index()
)


#### 🚨 Anomaly & Error Detection

1. Flag extreme `demand` values via z‑score.  
2. Use `IsolationForest` on all scaled continuous features.  
3. Optionally remove the anomalies to create `df_clean`.

In [51]:
# Z‑score anomalies
df_sample['demand_z'] = df_sample.groupby('city')['demand'].transform(zscore)
anoms_z = df_sample[np.abs(df_sample['demand_z']) > 3]

# IsolationForest anomalies
iso = IsolationForest(contamination=0.01, random_state=42)
df_sample['if_label'] = iso.fit_predict(df_sample[to_scale])
anoms_if = df_sample[df_sample['if_label'] == -1]

# Remove anomalies
df_clean = df_sample[df_sample['if_label'] == 1].copy()

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but IsolationForest was fitted with feature names
  warnings.warn(


,demand,city,summary,precipIntensity,precipProbability,temperature,humidity,pressure,windSpeed,uvIndex,...,period,timestamp,hour,dayofweek,month,season,date,week,demand_z,if_label
1,1.396698,dallas,Clear,-0.196337,-0.374245,1.317770,-0.386250,-0.489702,3.207678,3.290212,...,2020-05,2020-05-02 19:00:00,19,5,5,spring,2020-05-02,18,-0.104672,1
2,3.118086,dallas,Humid,-0.196337,-0.374245,1.756189,0.209040,-1.511175,1.694944,-0.584801,...,2019-06,2019-06-21 02:00:00,2,4,6,summer,2019-06-21,25,2.130158,1
3,1.618770,dallas,Clear,-0.134752,-0.306878,-1.076136,0.025874,1.553244,-0.339110,-0.154244,...,2019-11,2019-11-16 15:00:00,15,5,11,autumn,2019-11-16,46,0.183638,1
6,1.197568,dallas,Overcast,-0.028379,0.366795,-0.962261,1.353827,-0.457274,0.091808,-0.584801,...,2020-01,2020-01-23 06:00:00,6,3,1,winter,2020-01-23,4,-0.363197,1
7,0.604075,dallas,Mostly Cloudy,0.022008,0.770998,0.195469,1.308036,-1.657100,0.685451,-0.584801,...,2020-02,2020-02-04 12:00:00,12,1,2,winter,2020-02-04,6,-1.133712,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,-1.212982,seattle,Overcast,-0.196337,-0.374245,0.169531,0.254831,0.483130,0.613129,1.137427,...,2018-09,2018-09-08 20:00:00,20,5,9,autumn,2018-09-08,36,-0.316365,1
49996,-1.195234,seattle,Possible Light Rain,1.673591,3.465688,-0.544719,1.033287,-1.203112,1.767266,0.276313,...,2019-04,2019-04-03 17:00:00,17,2,4,spring,2019-04-03,14,0.090721,1
49997,-1.243068,seattle,Mostly Cloudy,-0.157146,-0.172144,-0.903425,0.758538,1.877521,-0.483754,-0.584801,...,2019-04,2019-04-10 07:00:00,7,2,4,spring,2019-04-10,15,-1.006427,1
49998,-1.258652,seattle,Clear,-0.134752,-0.306878,0.078431,0.575372,-0.327563,-0.993021,-0.584801,...,2019-08,2019-08-07 08:00:00,8,2,8,summer,2019-08-07,32,-1.363869,1


#### 💾 10. Build Final Preprocessed DataFrame & Save

Select only the required features—including our four time features— and export the cleaned dataset to the external drive.

In [52]:
final_cols = [
    'timestamp','city','temperature','humidity','windSpeed','demand',
    'hour','dayofweek','month','season'
]
for opt in ['pressure','precipIntensity','precipProbability']:
    if opt in df_clean.columns:
        final_cols.append(opt)

final_df = df_clean[final_cols].reset_index(drop=True)

print("Sample of final_df:")
display(final_df.head())
print("Final shape:", final_df.shape)

# Save to external volume
output_path = '/Volumes/meen/preprocessed_dataset.csv'
final_df.to_csv(output_path, index=False)
print("Saved to", output_path)


Sample of final_df:


,timestamp,city,temperature,humidity,windSpeed,demand,hour,dayofweek,month,season,pressure,precipIntensity,precipProbability
0,2020-05-02 19:00:00,dallas,1.317770,-0.386250,3.207678,1.396698,19,5,5,spring,-0.489702,-0.196337,-0.374245
1,2019-06-21 02:00:00,dallas,1.756189,0.209040,1.694944,3.118086,2,4,6,summer,-1.511175,-0.196337,-0.374245
2,2019-11-16 15:00:00,dallas,-1.076136,0.025874,-0.339110,1.618770,15,5,11,autumn,1.553244,-0.134752,-0.306878
3,2020-01-23 06:00:00,dallas,-0.962261,1.353827,0.091808,1.197568,6,3,1,winter,-0.457274,-0.028379,0.366795
4,2020-02-04 12:00:00,dallas,0.195469,1.308036,0.685451,0.604075,12,1,2,winter,-1.657100,0.022008,0.770998


Final shape: (41703, 13)
Saved to /Volumes/meen/preprocessed_dataset.csv


In [53]:
print(final_df.isna().sum())

timestamp            0
city                 0
temperature          0
humidity             0
windSpeed            0
demand               0
hour                 0
dayofweek            0
month                0
season               0
pressure             0
precipIntensity      0
precipProbability    0
dtype: int64
